In [8]:
#import packages
import numpy as np
import pandas as pd
from keras import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation, LSTM
import time
import datetime
import queue

In [6]:
from PyQt5.QtCore import pyqtSlot, pyqtSignal, QObject
import win32com.client
class Eventhandler:
    
    def a(self):
        print('a')
        
    @classmethod
    def get_instance(cls):
        cp_cur = win32com.client.DispatchWithEvents("dscbo1.StockCur", cls)
        return cp_cur

a = Eventhandler.get_instance()
a.a()

com_error: (-2147221164, '클래스가 등록되지 않았습니다.', None, None)

In [ ]:
#CREON API REALTIME
import sys
from PyQt5.QtCore import pyqtSlot, pyqtSignal, QObject
import win32com.client
class CpMarketEye:
    def Request(self, codes, rqField):
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if(bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음.")
            return False
        objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        objRq.SetInputValue(0, rqField)
        objRq.SetInputValue(1, codes)
        #objRq.BlockRequest() #Block mode가 뭐임?
        rqStatus = objRq.GetDibStatus()
        rqRet = objRq.GetDibMsg1()
        print('통신상태' , rqStatus, rqRet)
        if rqStatus != 0:
            return False
        
        #종목하나만 넣을거지만, codes자료형이 배열로 되야하는지 아닌지 몰라서 일단 넣음
        cnt = objRq.GetHeaderValue(2)
        for i in range(cnt):
            rpCode = objRq.GetDataValue(0, i)
            rpName = objRq.GetDataValue(1, i)
            rpTime = objRq.GetDataValue(2,i)
            rpCurPrice = objRq.GetDataValue(5,i)
            rpVol = objRq.GetDataValue()(6,i)
            print(rpCode, rpName, rpTime, rpCurPrice, rpVol)
        
        return True
    
class StockCur(QObject):
    receive_real_stockcur_signal = pyqtSignal(tuple)
    
    def set_params(self, client):
        self.client = client
    
    def get_real(self, stockcode):
        stockcode = 'A'+stockcode
        self.SetInputValue(0, stockcode)
        self.Subscribe()
    
    def stop_real(self, stockcode:str):
        stockcode = 'A'+stockcode
        self.SetInputValue(0, stockcode)
        self.Unsubscribe()
    
    def OnReceived(self):
        code = self.client.GetHeaderValue(0)
        name = self.client.GetHeaderValue(1)
        ttime = self.client.GetHeaderValue(18)
        price = self.client.GetHeaderValue(13)
        cVol = self.client.GetHeaderValue(17)
        buysell = str(self.client.GetHeaderValue(26)) # 호가방식 체결상태 1:매수, 2:매도
        
        #treshold 값을 시총을 이용해서 구해주는것도 나쁘지않을듯.
        if cVol*price < 1000000: #100만원 이하의 피라미 주문은 안받음.
            pass
        else:
            tick = (code, name, ttime, price, cVol, buysell)
            self.receive_real_stockcur_signal.emit(tick)
    
    @classmethod
    def get_instance(cls):
        cp_cur = win32com.client.DispatchWithEvents("dscbo1.StockCur", cls)
        return cp_cur
    
    
            

In [ ]:
#CREON API TRANSACTION
import win32com.client

#연결여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = ojbCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음.")
    exit()

#주문초기화
objTrade = win32com.client.Dispatch("CpTrade.CpTdUtil")
initCheck = objTrade.TradeInit(0)
if (initCheck != 0):
    print("주문 초기화 실패")
    exit()

acc = objTrade.AccountNum[0]
accFlag = objTrade.GoodsList(acc, 1)
print(acc, accFlag[0])
objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
objStockOrder.SetInputValue(0, "2") #2:매수 1:매도
objStockOrder.SetInputValue(1, acc)
objStockOrder.SetInputValue(2, accFlag[0])#두번째 인자값 : 1:주식, 2:선옵 16:eurex 64:해선
objStockOrder.SetInputValue(3, "A003540") #종목코드
objStockOrder.SetInputValue(4, 1) #매수수량
#objStockOrder.SetInputValue(5, 12400) #주문 단가 입력. 시장가에서 입력 필요X
objStockOrder.SetInputValue(7, "0") # 0:기본 1:IOC(미체결자동취소) 2:FOK(미체결수량 있으면 전체 취소, 체결안됨)
objStockOrder.SetInputValue(8, "03") # "01" 보통주문 / "03" 시장가주문

#매수 주문 요청
objStockOrder.BlockRequest()
rqStatus = objStockOrder.GetDibStatus()
rqRet = objStockOrder.GetDibMsg1()
print("통신상태", rqStatus, rqRet) # 통신상태 0 0040 모의투자 매수 주문이 완료되었습니다.(ordss.cststkord)
if rqStatus != 0:
    exit()


In [10]:
#HyperParameters

input_dim = 2 #입력하는 feature 개수
#time_interval = 8 #minute. n분 마다 n분 누적 신호 발생. 혹은 lstm cell에서 시계열의 갯수
min_transaction_amount = 1000000 # 100만원 미만이면 리시버에서 버퍼로 데이터 안보내게끔.
n_cell = 32 # LSTM셀의 개수
n_labels = 4 #모델의 결과 값 라벨 수 e.g. n=4, 라벨이 강하락, 약하락, 약상승, 강상승 신호 발생.
seq_length = 14 # 모델이 참고 할 이전 분봉의 개수. LSTM/RNN에서
lr = 0.003 # 학습률
keep_prob = 1.0 #dropout rate
n_stacked_layers = 1 # 모델의 레이어 개수
n_epochs = 1000 # 학습횟수. 얼리스타핑 넣을까말까
n_batch_size = 32 #학습 배치 사이즈
model_path = None

In [ ]:
# DataLoading on API
# API에서 필요한 데이터를 가져오는 부분.
# 트레이딩 알고리즘에 실시간으로 전송할 데이터를 가져오는 함수 필요
# 모델 학습 및 테스팅 데이터 가져오는 부분은 다른 코드에서 따로 가져오기...바랍니다... 아님 여기다 넣어도 되고..

class DataReciever:
    def __init__(self, stockCode, buffer):
        self.api = CreonAPI.get_instance()
        self.stockCode = stockCode
        self.buffer = buffer#??? 버퍼와 데이터리시버는 상호참조? 혹은 리시버 has 버퍼
    
    #데이터를 불러올 때마다 발생
    def on_LoadData(self):
        self.buffer.add(api.data???)#??? 이부분은 구체적으로 api가 어떻게 이벤트를 발생시키는지 모르겠네요 아직
        #버퍼로 데이터 전달할 때, 받아온 체결 데이터에서 threshold를 min_transaction_amount / 체결가 만큼 넣어주면 좋을거같아요.
        #이렇게 하면 100만원 이하로 구매한 매매체결은 버퍼에 안들어가서 노이즈를 줄일 수 있을것 같습니다.
        pass
    
    # 데이터 발생 시 마다 버퍼가 데이터를 자동으로 받음.
    #def emitData(self):

In [ ]:
# Data Preprocessing for Predicting
# 실제 트레이딩 시, API에서 받아온 실시간 데이터를 버퍼에 넣기 전까지 전처리 하는 과정
# 데이터가 버퍼에 들어가기 전에 여기서 다듬고 나서 보내준다.
# 데이터 리시버가 이 클래스를 소유하고, 다듬은 다음 버퍼로 쏴주면 될듯? 아니면 버퍼가 프리프로세서를 소유해도 될듯.

In [ ]:
# Data Preprocessing for Training
# API에서 받아온 대용량 데이터를 모델 학습을 위해 인풋과 라벨값이 있는 지도학습 데이터로 바꿔주는 부분은 다른 곳에서 직접 짜기.

In [ ]:
# Data Buffer
# API에서 받아온 데이터가 전처리 되어 저장되는 공간.
class DataBuffer:
    def __init__(self, stockcode):
        self.stockcode = stockcode
        self.q = queue.Queue(maxsize = seq_length)
        self.stockcur = StockCur.get_instance()
        self.stockcur.receive_real_stockcur_signal.connect(self.__receive_real_stockcur)
    
    #@pyqtSlot()
    def subscribe():
        stockcode = self.stockcode
        self.stockcur.get_real(stockcode)
        
    def add(self,data):
        #여기다가 데이터 전처리 부분 넣을까말까
        
        if self.q.full():#q가 가득차면
            self.q.get()# 하나 꺼내고
            self.q.put(data)# 신규 데이터 넣고
        else:
            self.q.put(data)
            
    #모인 데이터를 df 형식으로 쏴준다.
    def convert_to_DF(self):
        return pd.DataFrame(list(self.q.queue))
    
    @pyqtSlot(tuple)
    def __receive_real_stockcur(self, tick: tuple):
        self.add(str(tick))
    
    if __name__ == "__main__":
        _ = Buffer("005930")
        _.subscribe()
    #Queue를 만들고 Queue가 갱신(새 데이터가 들어옴)될 때 마다 모델에 데이터 전송

In [ ]:
# Data Seperating ( Training / Validating / Testing)
# 학습용 데이터 전처리 과정에서 받아온 데이터를 트레이닝 밸리데이팅 테스팅 데이터로 나누는 부분

from sklearn.model_selection import train_test_split
testSize = 0.3 #전체 데이터에서 테스트 데이터의 크기
x_train, x_test, y_train, y_test = train_test_split(preprocessed_Data, 
                                                    preprocessed_Data['Label'], 
                                                    test_size = testSize, 
                                                    random_state=13)


In [28]:
np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,
            np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T]).shape

(2, 14, 2)

In [49]:
# Model Training & Saving
# 모델 생성 및 학습. 결과 저장

"""모델 라벨값은 총 4개. 0,1,2,3 으로 라벨링하고 
0(강 하락)에 매도, 1(약 하락), 2(약 상승)에 홀드or관망, 3(강 상승)에 매수"""

from keras import metrics

model = Sequential()
for i in range(n_stacked_layers):
    model.add(LSTM(n_cell, input_shape=(seq_length, input_dim), activation='sigmoid',weights=None))
model.add(Dense(4, activation='softmax'))

model.compile(loss='mse', optimizer='adam', metrics=[metrics.categorical_accuracy])
#어떤 loss func, optimizer를 써야할지, 메트릭스를 써야할지 모르겠다. 일단 나중에 생각하기.
history = model.fit(x = np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,
            np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T]), 
                    y = np.asarray(([1,2,3,4],[1,2,3,4])), batch_size=n_batch_size, epochs = n_epochs, validation_split=0.3)
#history = model.fit(x = x_train, y = y_train, batch_size=n_batch_size, epochs = n_epochs, validation_split=0.3)
#밸리데이션셋은 이전 단계에서 햇음. 데이터를 트레인 테스트로 8:2로 나눈다음 8을 
#5.6 : 2.4로 트레인 : 밸릳 데이터로 나눔.

#모델 저장
model_path = f"./model_{datetime.datetime.now().strftime('%m%d_%H%M%S')}.h5"
model.save(model_path)
import matplotlib.pyplot as plt
plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

#model.evaluate()
#model.save()

Train on 1 samples, validate on 1 samples
Epoch 1/1000
1/1 [==============================] - 0s 341ms/step - loss: 6.2440 - categorical_accuracy: 0.0000e+00 - val_loss: 6.2374 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 9ms/step - loss: 6.2374 - categorical_accuracy: 0.0000e+00 - val_loss: 6.2319 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.2319 - categorical_accuracy: 0.0000e+00 - val_loss: 6.2262 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.2262 - categorical_accuracy: 0.0000e+00 - val_loss: 6.2203 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.2203 - categorical_accuracy: 0.0000e+00 - val_loss: 6.2145 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 8ms/step - loss: 6.2145 - categorical_acc

Epoch 49/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.9517 - categorical_accuracy: 1.0000 - val_loss: 5.9429 - val_categorical_accuracy: 1.0000
Epoch 50/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.9429 - categorical_accuracy: 1.0000 - val_loss: 5.9337 - val_categorical_accuracy: 1.0000
Epoch 51/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.9337 - categorical_accuracy: 1.0000 - val_loss: 5.9243 - val_categorical_accuracy: 1.0000
Epoch 52/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.9243 - categorical_accuracy: 1.0000 - val_loss: 5.9148 - val_categorical_accuracy: 1.0000
Epoch 53/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.9148 - categorical_accuracy: 1.0000 - val_loss: 5.9052 - val_categorical_accuracy: 1.0000
Epoch 54/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.9052 - categorical_accuracy: 1.0000 - val_loss: 5.8957 - val_categorical_accuracy: 1.0000
Epoch 55/1

Epoch 99/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7590 - categorical_accuracy: 1.0000 - val_loss: 5.7587 - val_categorical_accuracy: 1.0000
Epoch 100/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7587 - categorical_accuracy: 1.0000 - val_loss: 5.7583 - val_categorical_accuracy: 1.0000
Epoch 101/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7583 - categorical_accuracy: 1.0000 - val_loss: 5.7579 - val_categorical_accuracy: 1.0000
Epoch 102/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7579 - categorical_accuracy: 1.0000 - val_loss: 5.7575 - val_categorical_accuracy: 1.0000
Epoch 103/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7575 - categorical_accuracy: 1.0000 - val_loss: 5.7571 - val_categorical_accuracy: 1.0000
Epoch 104/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7571 - categorical_accuracy: 1.0000 - val_loss: 5.7566 - val_categorical_accuracy: 1.0000
Epoch

Epoch 149/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7511 - categorical_accuracy: 1.0000 - val_loss: 5.7510 - val_categorical_accuracy: 1.0000
Epoch 150/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7510 - categorical_accuracy: 1.0000 - val_loss: 5.7510 - val_categorical_accuracy: 1.0000
Epoch 151/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7510 - categorical_accuracy: 1.0000 - val_loss: 5.7510 - val_categorical_accuracy: 1.0000
Epoch 152/1000
1/1 [==============================] - 0s 9ms/step - loss: 5.7510 - categorical_accuracy: 1.0000 - val_loss: 5.7510 - val_categorical_accuracy: 1.0000
Epoch 153/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7510 - categorical_accuracy: 1.0000 - val_loss: 5.7509 - val_categorical_accuracy: 1.0000
Epoch 154/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7509 - categorical_accuracy: 1.0000 - val_loss: 5.7509 - val_categorical_accuracy: 1.0000
Epoc

Epoch 199/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7505 - categorical_accuracy: 1.0000 - val_loss: 5.7505 - val_categorical_accuracy: 1.0000
Epoch 200/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7505 - categorical_accuracy: 1.0000 - val_loss: 5.7505 - val_categorical_accuracy: 1.0000
Epoch 201/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7505 - categorical_accuracy: 1.0000 - val_loss: 5.7505 - val_categorical_accuracy: 1.0000
Epoch 202/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7505 - categorical_accuracy: 1.0000 - val_loss: 5.7505 - val_categorical_accuracy: 1.0000
Epoch 203/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7505 - categorical_accuracy: 1.0000 - val_loss: 5.7504 - val_categorical_accuracy: 1.0000
Epoch 204/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7504 - categorical_accuracy: 1.0000 - val_loss: 5.7504 - val_categorical_accuracy: 1.0000
Epoc

...

Epoch 849/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 850/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 851/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 852/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 853/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 854/1000
1/1 [==============================] - 0s 9ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoc

Epoch 899/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 900/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 901/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 902/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 903/1000
1/1 [==============================] - 0s 8ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 904/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoc

Epoch 949/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 950/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 951/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 952/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 953/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 954/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoc

Epoch 999/1000
1/1 [==============================] - 0s 6ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000
Epoch 1000/1000
1/1 [==============================] - 0s 7ms/step - loss: 5.7500 - categorical_accuracy: 1.0000 - val_loss: 5.7500 - val_categorical_accuracy: 1.0000


AttributeError: 'History' object has no attribute 'losses'

In [44]:
model.predict(np.asarray([np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T,
            np.asarray([[1,23,14,14,14,14,214,12,41,12,4,214,1,42],[1,23,14,14,14,14,214,12,41,12,4,214,1,42]]).T]))

array([[1.3848182e-04, 5.3457818e-05, 1.7756347e-03, 9.9803239e-01],
       [1.3848182e-04, 5.3457818e-05, 1.7756347e-03, 9.9803239e-01]],
      dtype=float32)

In [ ]:
# Model Testing(생략가능)
# 테스팅 데이터로 모델 테스트.
# 목표 Loss 값이 될 때 까지 전 단계(생성/학습) 반복
# 한 칸 위 코드를 모듈화해서 하이퍼파라미터 그리드 서치 후 최고 점수 모델 저장.

In [ ]:
# Trading with Trained Model
# 모델이 발생시킨 신호를 판단하여 API에 매매 요청.
class Trader:
    def __init__(self, logger):
        self.logger = logger
    
    def buy(self,):
        self.logger.addLog("buy", n_buy, buy_price, account_balance)
        pass
        
    def sell(self,):
        pass
    
    def action(self, sign):
        if sign == 0:
            if 계좌에 포지션이 존재하는 경우:
                self.sell()
            elif 계좌에 포지션이 없는 경우:
                pass # 로그 찍기("매도신호 발생. 포지션 없음." 이런식으로)
        elif sign == 3:
            if 계좌에 현금이 있는 경우:
                self.buy()
            elif 계좌에 현금이 없는 경우:
                pass
            

In [ ]:
# Logging
# 트레이딩 결과 로깅 및 기초통계량 작성
class Logger:
    def __init__(self):
        self.logs = []
    
    def addLog(self, *args)
        log = []
        for i in args:
            log.append(i)
        now = datetime.datetime.now()
        log = str(now) + str(log)
        self.logs.append(log)
        print(log)
    
    def saveLogs(self):
        with open(f"./logs_{datetime.datetime.now().strftime('%m%d_%H%M%S')}.txt", 'w') as f:
            for log in self.logs:
                f.write("%s\n" % log)
                  
    def statistics(self):
        pass

In [ ]:
# System Trading w/ Model(MAIN LOOP)

#Instance Init
buffer = DataBuffer() # 데이터 버퍼
dataReciever = DataReciever('005930', buffer) # API에서 데이터 리시버
model = model.load(model_path)#모델 로드
logger = Logger()
trader = Trader(logger)

#완성된 모듈을 인스턴스화 하여 변수에 할당하고
#각 각의 루틴(데이터루틴, 시그널루틴/매매루틴)이 인스턴스를 참조하여 트레이딩 구현

"""
루틴1 :리시버가 API로부터 데이터를 받아올 때마다 버퍼에 (발생한)데이터를 전송하는 코드를 짜야합니당..
이 부분을 비동기 처리 해준다음(이때 버퍼의 데이터는 이 루틴에서 조작됨), 버퍼의 데이터를 받아와서
모델에 넣고 """
async def a(n):
    await asyncio.sleep(1)
    pass

async def b():
    pass

async def c():
    pass

routines = [a(3), b(), c()]

async def main():
    await asyncio.wait(routines)


if __name__ == "__main__":
    asyncio.run(main())
    

## 프로그램 한계점
#### 1개 종목에 대해서만 감시하고 매매 할 수 있게 되어있음.
#### 나중에 필요시 알고리즘을 여러개 띄우거나 여러 종목에 대해서 트레이딩하도록 구현 필요
#### 트레이딩 종목 선정 구현 안되어있음
#### 매매수량 결정에 대한 알고리즘 없음
#### 모델이 핏 되지 않음.
#### 동호가 예외처리 되지않아서 9:00-15:19 까지만 사용 가능.
#### 데이터 피처 엔지니어링을 잘 안함..